# A script to retrieve weather data from Openweathermap API and export it into an HTML report
## Resources used:
### Open Weather Map API documentation:
- https://openweathermap.org/current
- https://openweathermap.org/api/one-call-api

### Converting UNIX timestamps to datetime:
- https://stackabuse.com/converting-strings-to-datetime-in-python/

### Plotly line chart documentation:
- https://plotly.com/python/line-charts/

### Plotly static image export:
- https://plotly.com/python/static-image-export/

In [1]:
import json
import requests
import time
from datetime import datetime as dt
import pandas as pd
import plotly.graph_objects as go
import os
if not os.path.exists("images"):
    os.mkdir("images")

In [2]:
# vispirms izmantojam "Current Weather Data API", meklējot pēc pilsētas nosaukuma
# JSON atbildē, pirmā atslēga ir "coords", jeb koordinātes, kuras ir nepieciešamas lai pieprasīt pilnu prognozi

def GetCoordinates(): 
    valid_input = False
    while valid_input == False:
        cityname = input("Please enter city name: ")
        req_coords = f"https://api.openweathermap.org/data/2.5/weather?q={cityname}&appid=c0a536afbc99f8d403c245f5f0a0158a"
        response = requests.get(req_coords)
        if response.status_code != 200:
            print(f"Error getting coordinates: {response.status_code}, {response.json().get('message')}")
        else:
            coords = response.json().get('coord')
            return coords,cityname

# Otrai funkcijai padodam koordinātes no pirmās funkcijas, tas ļaus pieprasīt pilnu prognozi
def GetWeatherData(coordinates):
    lon = coordinates['lon']
    lat = coordinates['lat']
    req_data = f"https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&exclude=minutely&appid=c0a536afbc99f8d403c245f5f0a0158a&units=metric"
    weather_data = requests.get(req_data)
    return weather_data
    

In [3]:
city_info = GetCoordinates() # fukcijas atgriež tuple ar koordinātēm un pilsētas nosaukumu
cityname = city_info[1] # saglabājam pilsētas nosaukumu, lai vēlāk to var izmantot atskaitē
forecast = GetWeatherData(city_info[0])

Please enter city name: New York


In [4]:
# Šajā solī apstrādājam datus

data = json.loads(forecast.text)
current = data['current']
hourly = data['hourly']
daily = data['daily']

# Laiks ir norādīts UTC laika zonā. Izmantojot atslēgu "timezone_offset" konvertējam laiku, lai tas atbilst pilsētas laika zonai
# Konvertējam datumus un laikus no UNIX timestamp formāta uz cilvēkam saprotamu

adjusted_time = current['dt']+forecast.json().get('timezone_offset')
adjusted_sunrise = current['sunrise']+forecast.json().get('timezone_offset')
adjusted_sunset = current['sunset']+forecast.json().get('timezone_offset')
current['dt'] = dt.utcfromtimestamp(adjusted_time).strftime('%Y-%m-%d %H:%M')
current['sunrise'] = dt.utcfromtimestamp(adjusted_sunrise).strftime('%H:%M')
current['sunset'] = dt.utcfromtimestamp(adjusted_sunset).strftime('%H:%M')

for i in range(0,len(hourly)):
    adjusted_time = hourly[i]['dt']+forecast.json().get('timezone_offset')
    hourly[i]['dt'] = dt.utcfromtimestamp(adjusted_time).strftime('%Y-%m-%d %H:%M')
    
for i in range(0,len(daily)):
    adjusted_time = daily[i]['dt']+forecast.json().get('timezone_offset')
    adjusted_sunrise = daily[i]['sunrise']+forecast.json().get('timezone_offset')
    adjusted_sunset = daily[i]['sunset']+forecast.json().get('timezone_offset')
    daily[i]['dt'] = dt.utcfromtimestamp(adjusted_time).strftime('%Y-%m-%d %H:%M')
    daily[i]['mintemp'] = daily[i]['temp']['min']
    daily[i]['maxtemp'] = daily[i]['temp']['max']
    

    
# Saglabājam apstrādātus datus JSON failos talākai apstrādei izmantojot pandas

with open("hourly.json", "w", encoding="utf-8") as f:
    json.dump(hourly, f, indent=4)
    
with open("daily.json", "w", encoding="utf-8") as f:
    json.dump(daily, f, indent=4)

In [5]:
# ar Pandas palīdzību uztaisam stundu un temperatūru sarakstus
h_data = pd.read_json (r'hourly.json')

hours = list(h_data["dt"])
temp = list(h_data["temp"])

# izmantojot izveidotus sarakstus var uzzimēt grafiku caur Plotly

fig = go.Figure()
fig.add_trace(go.Scatter(x=hours[:13], y=temp, hoverinfo='y', mode='lines',
                         line=dict(color='darkred', width=3)))
fig.update_layout(title='Hourly forecast for next 12 hours',yaxis_range=[-40,40], xaxis_title='Time', yaxis_title='Temperature °C',)
fig.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='LightBlue')
fig.write_image("images/hourly.jpeg")

In [6]:
# atkartojam procesu ilgtermiņa prognozei

d_data = pd.read_json (r'daily.json')

days = list(d_data["dt"])
min_temp = list(d_data["mintemp"])
max_temp = list(d_data["maxtemp"])

fig = go.Figure()
fig.add_trace(go.Scatter(x=days[:7], y=max_temp, name='Daily max °C', hoverinfo='y', mode='lines',
                         line=dict(color='darkred', width=4)))
fig.add_trace(go.Scatter(x=days[:7], y=min_temp, name='Daily min °C', hoverinfo='y', mode='lines',
                         line=dict(color='darkblue', width=4)))
fig.update_layout(title='Forecast for next 7 days',yaxis_range=[-40,40],
                  xaxis_title='Time', yaxis_title='Temperature °C',)
fig.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='LightBlue')
fig.write_image("images/daily.jpeg")

In [7]:
# eksportējam atskaiti HTML formātā

html = f'''
<!DOCTYPE html>
<html>
    <head>
        <title>{cityname} forecast</title>
        <link rel="stylesheet" type="text/css" href="style.css" />
    </head>
    <body>
        <div class="header"> 
            <h1>Weather in {cityname}</h1>
        </div>
        <div id="text_forecast">
            <h2>Current weather:</h2>
            <p>Date: {current['dt']}<p>
            <p>Sunrise: {current['sunrise']}</p>
            <p>Sunset: {current['sunset']}</p>
            <p>Temperature: {current['temp']}°C</p>
            <p>Feels like: {current['feels_like']}°C</p>
            <p>Ambient pressure: {current['pressure']}hPa</p>
            <p>Humidity: {current['humidity']}%</p>
        </div>
        <div id="hourly_forecast">
            <img src="images\hourly.jpeg">
        </div>
        <div id="daily_forecast">
            <img src="images\daily.jpeg">
        </div>
    </body>
</html>
'''

file = open("weather_report.html","w")
file.write(html)
file.close()